In [1]:
import pandas as pd

Veriyi okuyup verinin ne olduğuna bakalım. Veriyi tekrar gözden geçirelim.

In [4]:
data = pd.read_csv('data_for_preprocessing.csv')
df = data.copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6069 entries, 0 to 6068
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6069 non-null   int64 
 1   Text        6069 non-null   object
 2   Author      6069 non-null   object
dtypes: int64(1), object(2)
memory usage: 142.4+ KB


Boş veri var mı diye kontrol ediyorum. Bir de Unnamed: 0 columnu bir işime yaramayacak. Kaldırıyorum.

In [8]:
df.isna().sum()

Unnamed: 0    0
Text          0
Author        0
dtype: int64

In [13]:
df.drop(columns = 'Unnamed: 0', inplace = True)
df.head()

,Text,Author
0,This study investigates the chemical compositi...,AI
1,This study explores the cultural history of oi...,AI
2,Isolation of human peripheral blood mononucle...,Human
3,Dynamic Bayesian Networks (DBNs) are probabil...,Human
4,"Within volleyball, performance analysis is em...",Human


Model string değerleri anlayamadığı AI ve Human'ı 1 ve 0'a dönüştürüyorum. Bunun için label encoding de yapılabilir ama ben map yoluyla label_num dicem. Extra kütüphane dahil etmek istemiyorum.

In [16]:
df['label_num'] = df['Author'].map({'Human': 0, 'AI': 1})
df.head()

,Text,Author,label_num
0,This study investigates the chemical compositi...,AI,1
1,This study explores the cultural history of oi...,AI,1
2,Isolation of human peripheral blood mononucle...,Human,0
3,Dynamic Bayesian Networks (DBNs) are probabil...,Human,0
4,"Within volleyball, performance analysis is em...",Human,0


In [20]:
df.drop(columns = 'Author', inplace = True)
df.head()

,Text,label_num
0,This study investigates the chemical compositi...,1
1,This study explores the cultural history of oi...,1
2,Isolation of human peripheral blood mononucle...,0
3,Dynamic Bayesian Networks (DBNs) are probabil...,0
4,"Within volleyball, performance analysis is em...",0


Modelimizi bağımlı ve bağımsız değişken olarak atayalım.

In [25]:
from sklearn.model_selection import train_test_split

In [31]:
X = df.Text
y = df.label_num
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Artık verim eğitilebilir duruma geldi. Artık önişleme süreci için vectorizer ve stopwords silimi yapabiliriz. Bunun içinde Pipeline kullanıcaz. Veriyi eğitirken aynı zamanda önişleme yapabileceğiz. 4 farklı model kullanıcam.
1. Logistic Regression
2. SVM
3. Naive Bayes
4. Random Forest
   
Bunun için configurasyonlara başlayalım.

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [40]:
model_configs = [
    {
        'name': 'Logistic Regression',
        'file_name': 'model_lr.pkl',
        'step_name': 'lr',
        'model': LogisticRegression(),
        'params': {
            'vect__max_features': [5000],
            'vect__ngram_range': [(1, 2)], 
            'lr__C': [1, 10]
        }
    },
    {
        'name': 'SVM',
        'file_name': 'model_svm.pkl',
        'step_name': 'svc',
        'model': SVC(probability=True),
        'params': {
            'vect__max_features': [5000],
            'svc__C': [1, 10],
            'svc__kernel': ['linear']
        }
    },
    {
        'name': 'Naive Bayes',
        'file_name': 'model_nb.pkl',
        'step_name': 'nb',
        'model': MultinomialNB(),
        'params': {
            'vect__max_features': [5000],
            'nb__alpha': [0.1, 1.0]
        }
    },
    {
        'name': 'Random Forest',
        'file_name': 'model_rf.pkl',
        'step_name': 'rf',
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'vect__max_features': [5000],
            'rf__n_estimators': [100], # 100 tane ağaç diker
            'rf__max_depth': [None, 20] # Ağaç derinliği
        }
    }
]

Şimdi modelleri sırasıyla eğitip en iyi parametreleri seçip ona göre en başarılarını alıp yolumuza devam edicez. Yapalım. Stopwords için de pipelineımızı burada kullanıcaz.

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib # en son da modeli dosya olarak aktaralım.

In [48]:
for config in model_configs:
    print(config['name'] + ' eğitiliyor.')
    
    pipeline = Pipeline([
        ('vect', TfidfVectorizer(stop_words='english')),
        (config['step_name'], config['model'])
    ])
    
    clf = GridSearchCV(pipeline, config['params'], cv=3, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    acc = clf.best_score_
    print(f"  -> {config['name']} Başarısı: %{acc*100:.2f}")
    
    joblib.dump(clf.best_estimator_, config['file_name'])
    print(f"  -> Kaydedildi: {config['file_name']}\n")

Logistic Regression eğitiliyor.
  -> Logistic Regression Başarısı: %98.70
  -> Kaydedildi: model_lr.pkl

SVM eğitiliyor.
  -> SVM Başarısı: %98.74
  -> Kaydedildi: model_svm.pkl

Naive Bayes eğitiliyor.
  -> Naive Bayes Başarısı: %95.02
  -> Kaydedildi: model_nb.pkl

Random Forest eğitiliyor.
  -> Random Forest Başarısı: %98.91
  -> Kaydedildi: model_rf.pkl

